In [94]:
import numpy as np
import sys
import open3d as o3d
import pandas as pd
from scipy.spatial import Delaunay
from scipy.spatial.distance import cdist

from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.ResidueDepth import get_surface

def GridInPocket(AtomCoord, Surface):
    
    # 1.inital grid points
    X = np.arange(-15, 15, 1.375)
    Y = np.arange(-6, 6, 1.375)
    Z = np.arange(0, 12, 1.375)
    xx,yy,zz = np.meshgrid(X, Y, Z)
    grid_coord = np.hstack((xx.reshape(-1,1),yy.reshape(-1,1),zz.reshape(-1,1)))

    # 2. protruding grid points
    box = Delaunay(AtomCoord)
    InGroove = box.find_simplex(grid_coord)>=0

    # 3. crash grid points
    NoCrash = cdist(grid_coord, AtomCoord).min(axis=1) > 1.375
    
    return grid_coord[InGroove * NoCrash]

def ShowAtomCloud(DATFile, PDBfile):
    
    DAT = pd.read_csv(DATFile)
    Coord = DAT[['X', 'Y', 'Z']].values

    parser = PDBParser(PERMISSIVE=1)
    Struct = parser.get_structure("struct", PDBfile)[0]
    Surface = get_surface(Struct, MSMS="/home/shawn/local/msms_i86_64Linux2_2.6.1/msms.x86_64Linux2.2.6.1")

    P1 = o3d.geometry.PointCloud()
    P2 = o3d.geometry.PointCloud()
    P3 = o3d.geometry.PointCloud()
    
    P1.points = o3d.utility.Vector3dVector(Coord)
    P2.points = o3d.utility.Vector3dVector(GridInPocket(Coord, Surface))
    P3.points = o3d.utility.Vector3dVector(Surface)
    # P1.colors = o3d.utility.Vector3dVector(color1)
    # P2.colors = o3d.utility.Vector3dVector(color2)

    P3.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=2, max_nn=10))
    print(P3.normals)
    sys.exit()
    # radii = [1, 2, 4]
    # rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(P3, o3d.utility.DoubleVector(radii))
    alpha = 1.375
    rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(P3, alpha)

    # rec_mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(P3, depth=9)

    o3d.visualization.draw_geometries([P3, rec_mesh],point_show_normal=True)

    P1.paint_uniform_color([1, 0, 0])
    P2.paint_uniform_color([0, 1, 0])
    # P3.paint_uniform_color([0, 0, 1])
    # o3d.visualization.draw_geometries([P1, P2])

    return

ShowAtomCloud("../crystal/A_mean/DAT/A01_01.csv", "../crystal/A_mean/pdb/A01_01.pdb")

std::vector<Eigen::Vector3d> with 10575 elements.
Use numpy.asarray() to access data.


SystemExit: 

/home/shawn/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [89]:
msmsout = "A01_01.vert"

with open(msmsout) as fp:
    vertex_list = []
    norm_list = []
    for l in fp:
        sl = l.split()
        if len(sl) != 9:
            # skip header
            continue
        vl = [float(x) for x in sl[0:3]]
        nl = [float(x) for x in sl[3:6]]
        vertex_list.append(vl)
        norm_list.append(nl)

vertex_list = np.array(vertex_list)
norm_list = np.array(norm_list)

In [96]:
P1 = o3d.geometry.PointCloud()
P1.points = o3d.utility.Vector3dVector(vertex_list)
P1.normals = o3d.utility.Vector3dVector(norm_list)
o3d.visualization.draw_geometries([P1],point_show_normal=True)

In [42]:
A = np.array([[1,1,1],[2,2,2],[3,3,3]])
B = np.array([True,False,False])
C = np.array([True,True,False])
B*C

array([ True, False, False])